## 自己构建的网络

In [1]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
from keras.layers import Lambda
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input as resnet50_pre
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input as inceptionV3_pre
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input as xception_pre
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input as vgg16_pre
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input as vgg19_pre
from keras.preprocessing import image   
from keras.optimizers import SGD, Adam
from keras.utils.np_utils import to_categorical
from tqdm import tqdm
from PIL import ImageFile  
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, Input, Activation
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint  
import numpy as np
import pandas as pd
from glob import glob
import cv2                
import matplotlib.pyplot as plt    
import matplotlib.image as mpimg
%matplotlib inline 
import random
import os
import shutil
import h5py
import common

Using TensorFlow backend.


## 对图片进行分类

In [3]:
common.divide_images()

In [2]:
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

W0325 14:33:24.044587 15340 deprecation.py:506] From C:\ProgramData\Anaconda3\envs\graduate\lib\site-packages\tensorflow\python\training\moving_averages.py:210: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
def batch_extract_features():
    models_and_features = {}
    
    # VGG16
    base_model = VGG16(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_features(base_model, (224, 224), vgg16_pre)
    models_and_features['vgg16'] = features_name
    
    # VGG19
    base_model = VGG19(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_features(base_model, (224, 224), vgg19_pre)
    models_and_features['vgg19'] = features_name
    
    # ResNet50
    base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_features(base_model, (224, 224), resnet50_pre)
    models_and_features['resnet50'] = features_name
    
    # InceptionV3
    base_model = InceptionV3(weigths='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_features(base_model, (299, 299), inceptionV3_pre)
    models_and_features['inceptionV3'] = features_name
    
    # Xception
    base_model = Xception(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_features(base_model, (299, 299), xception_pre)
    models_and_features['xception'] = features_name
    
    return models_and_features

In [ ]:
models_and_features = batch_extract_features()

for key, value in models_and_features.items():
    features = np.load(value)
    train_features = features['train']
    train_labels = features['train_label']
    valid_features = features['valid']
    valid_labels = features['valid_label']
    test_features = features['test']
    test_filenames = resnet50_features['test_filename']
    
    model = Sequential()
    model.add(Dense(2048, input_shape=(train_features.shape[1],), activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(2048, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    
    checkpointer = ModelCheckpoint(filepath='{0}.hdf5'.format(key), verbose=1, save_best_only=True)
    model.fit(train_features, train_labels, epochs=2, batch_size=batch_size,
          validation_data=(valid_features, valid_labels), verbose=1, callbacks=[checkpointer])
    
    model.load_weights('{0}.hdf5'.format(key))
    prediction = model.predict(test_features, batch_size=common.batch_size)
    prediction = prediction[:, 0].clip(0.01, 0.99)
    test_fileindex = np.array([os.path.splitext(os.path.split(filename)[1])[0] for filename in test_filenames])
    data = np.stack([test_fileindex, prediction], axis=1)
    pd.DataFrame(data, columns=['id', 'label']).to_csv('{0}_predict.csv'.format(key), index=False)
    